# Using Cloud Datalab - Accessing Cloud Data

This notebook describes how Cloud Datalab integrates within your Google Cloud project, and how you can work with data, manage your notebooks, and invoke APIs that are part of Google Cloud Platform.

## An Under-the-Covers Look

The Cloud Datalab functionality is packaged into a docker container. This container contains a ready-to-use environment including the Python runtime, a set of libraries picked for data analysis and visualization scenarios, Google Cloud Platform integration functionality and this front-end server enabling this environment.

Each deployed or running Cloud Datalab environment represents a Cloud Datalab _workspace_. A workspace is comprised of two parts:

* A named version (by default `main`) of an AppEngine module called `datalab`. AppEngine launches a separate Compute Engine instance (or [managed VM](https://cloud.google.com/appengine/docs/managed-vms/)) for each version.

* A [source repository](https://cloud.google.com/tools/cloud-repositories/docs/) branch within the git repository associated with the Cloud project.

You can deploy one or more Cloud Datalab workspaces within your Cloud project. These workspace instances are accessible to all owners and editors within the Cloud project, and otherwise inaccessible to others. You might decide to use and share a single workspace, or you might decide to create dedicated instances - for example, one per user, or one per feature/workstream. The choice is yours.

Within this workspace, the Cloud Datalab front-end manages notebooks, notebook sessions and the corresponding instances of IPython and Python runtime.

## Google Cloud Integration

In [ ]:
import gcp

context = gcp.Context.default()
print 'The current project is %s' % context.project_id

Within Cloud Datalab, the `gcp` Python library provides access to Google Cloud Platform services. It automatically handles initialization to detect the current project, as well as the OAuth token to be used to invoke APIs. In particular it uses the OAuth token representing the project's service account, rather than an individual users.

## Service Accounts

This is an important detail.

The code you author and the data you access is stored in notebooks that are shared across the project. As such the authorization used to execute and retrieve that data is based on the project.

Additionally, any applications or data pipelines you produce within Cloud Datalab are deployed using the service account rather than individual accounts, which is generally good practice.

Correspondingly if there are resources contained within another project, you will need to authorize the service account of this project, rather than a particular user.

In [ ]:
context.credentials.get_access_token()

The above code prints out the access token representing the service account. However it is more useful to see the service account itself, so it can be authorized to resources and data in another project.

This service account is listed in the Developer Console for your project. Specifically within the Developer, navigate to the Permissions section. you should see the service account listed. There should be a service account with a `gserviceaccount.com` email id.

Often there are multiple service accounts. The one being used for within the Cloud Datalab environment can be retrieved using the following curl command (which invokes the [Compute Engine metadata service](https://cloud.google.com/compute/docs/metadata?hl=en)):

In [ ]:
%%bash
curl --silent -H "Metadata-Flavor: Google" \
  http://metadata/computeMetadata/v1/instance/service-accounts/default/email